In [3]:
using Pkg
Pkg.activate(".")

  Activating new project at `~/Desktop/juliaProject/main/notebook`


Running single threads

In [3]:
module main
using JLD2
using net

@load "../../data/imdb_dataset_prepared_embedings.jld2" X_train y_train X_test y_test embeddings vocab

X_train = Int32.(X_train)
y_train = Float32.(vec(y_train))
X_test = Int32.(X_test)
y_test = Float32.(vec(y_test))

chain = Chain(
    Embedding(embeddings),
    Convolution(3, size(embeddings, 1), 8, relu),
    MaxPool(8),
    Flatten(),
    Dense(128, 1, σ),
    Classification(binarycrossentropy)
)


startNetwork(X_train, y_train, X_test, y_test, chain, 64, 5)

end

Epoch: 1, time: 26.985613509, Loss: 0.5697293, Accuracy: 0.69465, LossTest: 0.4165767, AccuracyTest: 0.8122
Epoch: 2, time: 10.925329537, Loss: 0.35401136, Accuracy: 0.84615, LossTest: 0.33655655, AccuracyTest: 0.8528
Epoch: 3, time: 10.884208945, Loss: 0.26947704, Accuracy: 0.891125, LossTest: 0.31812206, AccuracyTest: 0.8613
Epoch: 4, time: 10.862917571, Loss: 0.20914106, Accuracy: 0.921125, LossTest: 0.3164333, AccuracyTest: 0.8663
Epoch: 5, time: 10.830551286, Loss: 0.15533797, Accuracy: 0.94715, LossTest: 0.33569756, AccuracyTest: 0.8681


Main.main

In [2]:
using JLD2
X_train = load("../../data/imdb_dataset_prepared_embedings.jld2", "X_train")
y_train = load("../../data/imdb_dataset_prepared_embedings.jld2", "y_train")
X_test = load("../../data/imdb_dataset_prepared_embedings.jld2", "X_test")
y_test = load("../../data/imdb_dataset_prepared_embedings.jld2", "y_test")
embeddings = load("../../data/imdb_dataset_prepared_embedings.jld2", "embeddings")
vocab = load("../../data/imdb_dataset_prepared_embedings.jld2", "vocab")

embedding_dim = size(embeddings, 1)

using Flux
using Printf, Statistics

model = Chain(
    Flux.Embedding(12849, embedding_dim),
    x -> permutedims(x, (2, 1, 3)),
    Conv((3,), embedding_dim => 8, relu),
    MaxPool((8,)),
    Flux.flatten,
    Dense(128, 1, σ)
)

model.layers[1].weight .= embeddings;


loss(m, x, y) = Flux.Losses.binarycrossentropy(m(x), y)
accuracy(m, x, y) = mean((m(x) .> 0.5) .== (y .> 0.5))

dataset = Flux.DataLoader((X_train, y_train), batchsize=64, shuffle=true)

opt = Optimisers.setup(Adam(), model)
epochs = 5
for epoch in 1:epochs
    total_loss = 0.0
    total_acc = 0.0
    num_samples = 0

    t = @elapsed begin
        for (x, y) in dataset
            grads = Flux.gradient(model) do m
                loss(m, x, y)
            end
            Optimisers.update!(opt, model, grads[1])
            total_loss += loss(model, x, y)
            total_acc += accuracy(model, x, y)
            num_samples += 1
        end

        train_loss = total_loss / num_samples
        train_acc = total_acc / num_samples

        test_acc = accuracy(model, X_test, y_test)
        test_loss = loss(model, X_test, y_test)
    end

    println(@sprintf("Epoch: %d (%fs) \tTrain: (l: %f, a: %f) \tTest: (l: %f, a: %f)",
        epoch, t, train_loss, train_acc, test_loss, test_acc))
end

Epoch: 1 (43.047512s) 	Train: (l: 0.544829, a: 0.717775) 	Test: (l: 0.398000, a: 0.827000)
Epoch: 2 (16.322143s) 	Train: (l: 0.334099, a: 0.858250) 	Test: (l: 0.335723, a: 0.857800)
Epoch: 3 (14.682731s) 	Train: (l: 0.255252, a: 0.897750) 	Test: (l: 0.323455, a: 0.864200)
Epoch: 4 (15.670740s) 	Train: (l: 0.195805, a: 0.928000) 	Test: (l: 0.324911, a: 0.869000)
Epoch: 5 (14.648823s) 	Train: (l: 0.142523, a: 0.952850) 	Test: (l: 0.342031, a: 0.868400)
